In [ ]:
seq = "M S S S N T D N Q Y P K Y I N D T T P P T I T L K E Y D N A S W A S T T C L D H N P I K N Q Y I V V V M E N P N Q I V A I I D Q Q D N M I L D I L F K N A H D A H S K Q E Y S T K"

In [ ]:
from transformers import BertModel, BertTokenizer
import re
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
seq = re.sub(r"[UZOB]", "X", seq)
encoded_input = tokenizer(seq, return_tensors='pt')


In [ ]:
import os
import math
import numpy as np
import random
import logging

# Bring in PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
# Most of the examples have typing on the signatures for readability
from typing import Optional, Callable, List, Tuple
from Bio import SeqIO
# For data loading
from torch.utils.data import Dataset, IterableDataset, TensorDataset, DataLoader
import json
import glob
import gzip
import bz2

# For progress and timing
from tqdm import tqdm
import time
import shutil
from Bio.PDB import PDBList
from Bio.PDB.MMCIFParser import MMCIFParser

In [ ]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )

seq_example = re.sub(r"[UZOB]", "X", seq_example)
encoded_input = tokenizer(seq_example, return_tensors='pt')


In [ ]:
def transformer(input_sequence, feed_forward_dim, output_dim=2):
    #putting the input in the embedding space
    model = BertModel.from_pretrained("Rostlab/prot_bert")
    
    output = model(**input_sequence)
    embedding_prot_bert = output.last_hidden_state
    N , D = embedding_prot_bert.size()[1], embedding_prot_bert.size()[2]

    #self, single-head, unmasked attention layer
    #attention weights
    w_q = nn.Parameter(torch.randn([D, D])) #(DxD)
    w_k = nn.Parameter(torch.randn([D, D])) #(DxD)
    w_v = nn.Parameter(torch.randn([D, D])) #(DxD)

    Q = torch.matmul(embedding_prot_bert, w_q)
    K = torch.matmul(embedding_prot_bert, w_k)
    V = torch.matmul(embedding_prot_bert, w_v)

    scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size()[1])
    #apply the softmax function to obtain the attention weights
    attn = torch.softmax(scores, dim=-1)
    # Compute the context vector as the weighted sum of the values V
    attention_output = torch.matmul(attn, V) #(NxD)

    #first normalization layer
    layer_norm1 = nn.LayerNorm((N,D))
    embedded_after_attention = layer_norm1(attention_output)

    #feed forward neural network
    feed_forward = nn.Sequential(
        nn.Linear(D, feed_forward_dim),
        nn.ReLU(),
        nn.Linear(feed_forward_dim, feed_forward_dim),
        nn.ReLU(),
        nn.Linear(feed_forward_dim, output_dim) 
        )
    
    ff_output = feed_forward(embedded_after_attention) #(Nx2)

    #second normalization layer
    layer_norm2 = nn.LayerNorm((N,2))
    embedded_after_ff = layer_norm2(ff_output)

    return embedded_after_ff





In [ ]:
input_sequence = encoded_input
output = transformer(input_sequence,500)
print(output)